In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import random


# empty data frame with columns

In [ ]:
apple_forum = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[], 'model':[]})

In [ ]:
model = {'iphone_X':str(1889), 'iphone_7':str(1849),'iphone_7_p':str(1850),
             'iphone_8':str(1890),'iphone_8_p':str(1891),'iphone_se':str(1847)}

In [ ]:
leng = 0
for model_name in model:
    leng = len(apple_forum)
    for i in range(1,28):
        html = requests.get("https://discussions.apple.com/community/iphone/iphone_hardware?page="+str(i)+"&category="+model[model_name]).text
        soup = BeautifulSoup(html, 'html.parser')
        bs_object = soup.find_all('div', attrs={"class":u"discussion-description"})

        for s,index in zip(bs_object,range(len(bs_object))): 
        #links to threads
            apple_forum.loc[index+15*(i-1)+leng,'link'] = s.find('div', attrs={"class":u"discussion-title"}).find('a').get('href')
    
        #question text
            apple_forum.loc[index+15*(i-1)+leng,'title'] = s.find('div', attrs={"class":u"discussion-title"}).find('a') \
            .find('span', attrs={"class":u"discussion-description-inner-responsive u-hide--desktop"}).get_text().strip()
    
        #view number
            apple_forum.loc[index+15*(i-1)+leng,'nb_views'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
            .find('div', attrs={"class":u"cell discussion-views u-hide--small"}).get_text().strip()
    
        #user name
            apple_forum.loc[index+15*(i-1)+leng,'user'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
            .find('div', attrs={"class":u"cell discussion-name u-hide--small"}).get_text().strip()
    
        #latest reply and user replying
            apple_forum.loc[index+15*(i-1)+leng,'date'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
            .find('div', attrs={"class":u"cell align-metadata"}).get_text().strip()
    
        #latest reply and user replying
            apple_forum.loc[index+15*(i-1)+leng,'nb_replies'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
            .find_all('div', attrs={"class":"cell align-metadata"})[1].get_text().strip()
        
        #model name
            apple_forum.loc[index+15*(i-1)+leng,'model'] = model_name
        
        print(model_name,i)
        time.sleep(random.random()*5)
   

In [ ]:
#apple_forum.to_csv('group_X.csv')
#apple_forum = apple_forum.drop(["index","level_0"],axis=1)
apple_forum = apple_forum.replace(r'\n',' ', regex=True)

In [ ]:
apple_forum.to_csv("group10_applesupport.csv")

In [ ]:
apple_forum = pd.read_csv("group10_applesupport.csv",index_col=0)

In [ ]:
apple_forum.link[:10]

In [ ]:
messages = pd.DataFrame({'thread_id' : [], 'date': [], 'message_id':[], 'message_text':[], \
                         'poster':[], 'interest':[], 'question':[], 'model':[]})

i=0
test = apple_forum.link
for link in test:
    html = requests.get("https://discussions.apple.com"+link).text
    soup = BeautifulSoup(html, 'html.parser')
    bs_object = soup.find('div', attrs={"class":u"j-column-wrap-l"})
    
    try:
        length = len(bs_object.find_all('div', attrs={"class":u"jive-rendered-content"}))
    except: length = 0
    l=0
    while l < length:
        if l == 0:
            messages.loc[i,'question'] = True
            
            ### poster name 
            messages.loc[i,'poster'] = bs_object.find('div', attrs={"class":u"j-post-author"}).find('a').text.strip()
            
            ###question text 
            #messages.loc[i,'message_text'] = bs_object.find('span',attrs = {'class':u'question-more js-toggle-title-content'}).text.strip()
            messages.loc[i,'message_text'] = bs_object.find('section',attrs = {'class':u'j-original-message'}) \
                                            .find('div',attrs = {'class':u'jive-rendered-content'}).text.strip()

            ### poster model name / OS
            try:
                messages.loc[i,'model'] = bs_object.find('p', attrs={"class":u"jive-model-info-display"}).text.strip()
            except: 
                messages.loc[i,'model'] = "not disclosed"
            ### poste date
            messages.loc[i,'date'] = bs_object.find('p', attrs={"class":u"meta-posted"}).text.strip()

            ###me too
            messages.loc[i,'interest'] = bs_object.find('a', attrs={"class":u" jive-acclaim-likelink black-button "}).text.strip()
            
            ###question id
            messages.loc[i,'message_id'] = bs_object.find('', attrs={"class":u" jive-acclaim-likelink black-button "}).get("data-object-id")
            
        else:
            messages.loc[i,'question'] = False
             
            permalink = bs_object.find_all('a', attrs={"class":u"discussionPermaLink"})
            try:
                messages.loc[i,'message_id'] = permalink[l-1].get("href")
            except:
                messages.loc[i,'message_id'] = "na"

            #user 
            users = bs_object.find_all('span', attrs={"class":u"username"})
            try:
                messages.loc[i,'poster'] = users[l-1].text.strip()
            except:
                messages.loc[i,'poster'] = "na"

            #replying to + date
            dates = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
            try:
                messages.loc[i,'date'] = dates[l-1].text.strip()
            except: 
                messages.loc[i,'date'] = "na"
            
            #helpful answer count
            helpful = bs_object.find_all('span', attrs={"class":u"js-acclaim-container acclaim-container acclaim-like-container "})
            
            try:
                messages.loc[i,'interest'] = helpful[l-1].get("data-likes")
            except:
                messages.loc[i,'interest'] = "na"
            
            #message text
            answer = bs_object.find_all('section')
            for x in answer:
                if x.find('span', attrs={"class":u"answer-marker"}):
                    continue
                else:
                    try: 
                        messages.loc[i,"message_text"] = answer[l-1].text.strip()
                    except:
                        messages.loc[i,"message_text"] = "na"
            
        messages.loc[i,'thread_id'] = link
       #m_text = bs_object.find_all('div', attrs={"class":u"jive-rendered-content"})
       #messages.loc[i,"message_text"] = m_text[l].text.strip()
        i+=1
        l+=1
        
messages.to_csv("group10_applemessages.csv")

In [ ]:
messages.shape

In [ ]:
bs_object.find('span', attrs={"class":u"jive-model-info-display"})

In [ ]:
html = requests.get("https://discussions.apple.com/thread/8163518").text
soup = BeautifulSoup(html, 'html.parser')
bs_object = soup.find('div', attrs={"class":u"j-column-wrap-l"})

In [ ]:
### poster name 
bs_object.find('div', attrs={"class":u"j-post-author"}).find('a').text.strip()
    
### poster model name / OS
bs_object.find('p', attrs={"class":u"jive-model-info-display"}).text.strip()

### question text
bs_object.find('span',attrs = {'class':u'question-more js-toggle-title-content'}).text.strip()
bs_object.find('section',attrs = {'class':u'j-original-message'}).find('div',attrs = {'class':u'jive-rendered-content'}).text
### poste date
bs_object.find('p', attrs={"class":u"meta-posted"}).text.strip()

###me too
bs_object.find('a', attrs={"class":u" jive-acclaim-likelink black-button "}).text.strip()

###question id
bs_object.find('', attrs={"class":u" jive-acclaim-likelink black-button "}).get("data-object-id")

### replying users
table = bs_object.find_all('span', attrs={"class":u"username"})
for x in table:
    x.text.strip()
    
#initial message + reply text: 
table = bs_object.find_all('div', attrs={"class":u"jive-rendered-content"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

#Replies only (exception for marked answers)
table = bs_object.find_all('section')
for x in table:
        if x.find('span', attrs={"class":u"answer-marker"}):
            continue
        else:
            (x)
    
#replying to
table = bs_object.find_all('span', attrs={"class":u"font-color-meta-light j-thread-replyto"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

##perma link to replies
table = bs_object.find_all('a', attrs={"class":u"discussionPermaLink"})
for x in range(0,len(table)):
    print(x)
    print(table[x].get("href"))

#replying to + date
table = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())
    
# helpful
table = bs_object.find_all('span', attrs={"class":u"js-acclaim-container acclaim-container acclaim-like-container "})
for x in table:
    print(x.get("data-likes"))

In [ ]:
#replying to + date
table = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

In [ ]:
bs_object.find('section',attrs = {'class':u'j-original-message'}).find('div',attrs = {'class':u'jive-rendered-content'}).text

In [ ]:
print(bs_object.prettify())

In [ ]:
answer = bs_object.find_all('section')
            for x in answer:
                if x.find('span', attrs={"class":u"answer-marker"}):
                    continue
                else:
                    messages.loc[i,"message_text"] = answer[l-1].text.strip()

In [89]:
#read csv locally (on github)
apple_forum = pd.read_csv("group10_applesupport.csv",index_col=0)

import nltk
# télécharger les packages de nltk, il faut wordnet, punkt 
#run pour l'interface
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


KeyboardInterrupt: 

In [ ]:
titles = apple_forum.title
titles[0] = "null"
for i in range(len(titles)):
    titles[i]=titles[i].lower()
    titles[i]=titles[i].replace("iphone x","iphone_x")
    titles[i]=titles[i].replace("iphone 8","iphone_7")
    titles[i]=titles[i].replace("iphone se","iphone_se")
    titles[i]=titles[i].replace("iphone 7","iphone_7")


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

tokens = titles.apply(nltk.word_tokenize,1)

for i in range(len(tokens)):
    tokens[i] = [word for word in tokens[i] if word.isalpha()]

stop_words = set(stopwords.words('english'))
    
for i in range(len(tokens)):
    tokens[i] = [w for w in tokens[i] if not w in stop_words]

tokens   


In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

stemmed = tokens.copy()
for i in range(len(stemmed)):
    stemmed[i] = [porter.stem(word) for word in stemmed[i]]
    
lemmatized = tokens.copy()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
for i in range(len(lemmatized)):
    lemmatized[i] = [wordnet_lemmatizer.lemmatize(word) for word in lemmatized[i]]

lemmatized

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=300, ngram_range=(1, 3))

tfidf_matrix = tfidf.fit_transform(lemmatized.transform(lambda x: ' '.join(x)))

tfidf.get_feature_names()

#tfidf_matrix
